In [2]:
import numpy as np
import pandas as pd
import netCDF4 as nc
import os
import shutil
import re
import xarray as xr
import spectral
import glob
import matplotlib.pyplot as plt
import math

In [13]:
from typing import Union
import pandas as pd
import numpy as np
import xarray as xr

'''
Sensor band configurations. Currently the BNNs are available for OLCI, MSI S2A/B and at some point will be Landsat-8 OLI.

'''

sensor_bands = {
'OLCI_all': ['413', '443', '490','510', '560', '620', '665', '673', '681', '708', '753','778'],
'OLCI_c2rcc': ['rhow_2', 'rhow_3', 'rhow_4', 'rhow_5','rhow_6','rhow_7','rhow_8', 'rhow_9', 'rhow_10', 'rhow_11', 'rhow_12', 'rhow_16'],
'OLCI_c2rcc_rrs': ['Rrs_2', 'Rrs_3', 'Rrs_4', 'Rrs_5','Rrs_6','Rrs_7','Rrs_8', 'Rrs_9', 'Rrs_10', 'Rrs_11', 'Rrs_12','Rrs_16'],
'OLCI_polymer_insitu': ['413', '443', '490','510', '560', '620', '665', '681', '708', '753','778'], # does not include 673 nm
'OLCI_polymer_rw': ['Rw413', 'Rw443', 'Rw490','Rw510', 'Rw560', 'Rw620', 'Rw665', 'Rw681', 'Rw708', 'Rw753','Rw778'], # does not include 673 nm
'MSI_s2a': ['443', '492', '560','665', '704', '740', '783'],
'MSI_s2b': ['443', '492', '560','665', '704', '739', '780'],
'L8_oli': ['443','482','561','655']
}


scaler_bands = {
'OLCI_all': ['413_res_s3b','443_res_s3b','490_res_s3b','510_res_s3b','560_res_s3b','620_res_s3b', '665_res_s3b','673_res_s3b','681_res_s3b','708_res_s3b','753_res_s3b','778_res_s3b'],
'OLCI_polymer':['413_res_s3b','443_res_s3b','490_res_s3b','510_res_s3b','560_res_s3b','620_res_s3b', '665_res_s3b','681_res_s3b','708_res_s3b','753_res_s3b','778_res_s3b'], # does not include 673 nm
'MSI_s2a':['443_res_s2a', '492_res_s2a', '560_res_s2a','665_res_s2a', '704_res_s2a','740_res_s2a', '783_res_s2a'],
'MSI_s2b':['443_res_s2b', '492_res_s2b','560_res_s2b','665_res_s2b', '704_res_s2b', '739_res_s2b','780_res_s2b']
}

def get_scaler_config(scaler_name):
    '''
    Get scaler configuration for the normalisation of the BNNs.
    
    Args:
        sensor_name : a sensor string of ['OLCI_all', 'OLCI_polymer', 'MSI_s2a', 'MSI_s2b'].
    
    Returns current sensor configurations for Sentinel-3 OLCI and Sentinel-2.
    '''
    scaler_config = sensor_bands[scaler_name]
    return scaler_config

def get_sensor_config(sensor_name):
    """
    Get sensor configuration for the functions and the BNNs and modify the dataset if necessary.
    
    Args:
        sensor_name (str): A sensor string, one of ['OLCI_all', 'OLCI_c2rcc', 'OLCI_polymer_rw', 'OLCI_polymer_insitu', 'MSI_s2a', 'MSI_s2b'].
        dataset (xr.Dataset or pd.DataFrame): The dataset to modify if necessary.
    
    Returns:
        xr.Dataset or pd.DataFrame: The possibly modified dataset.
    """
    sensor_config = sensor_bands[sensor_name]
        
    return sensor_config

def calculate_rrs(dataset, sensor_config):
    """
    Calculate remote sensing reflectance if necessary, e.g. for C2RCC or POLYMER.
    
    Args:
        dataset (xr.Dataset or pd.DataFrame): The dataset to modify.
        sensor_config (list): The configuration of the sensor bands.
        
    Returns:
        xr.Dataset or pd.DataFrame: The modified dataset.
    """
    modified_dataset = dataset.copy()
    for band in sensor_config:
        new_band_name = 'Rrs_' + band.split('_')[1]
        modified_dataset[new_band_name] = dataset[band] / np.pi
        print('Band ' + band + ' was divided by pi and stored as ' + new_band_name + '.')
    
    return modified_dataset

In [ ]:
# 1. filter all with idepix for valid pixels
# 2. apply owt classification
# 3. apply owt flagging
# 4. apply normalisation
# 5. calculate bnn chla and bnn unc

In [ ]:
# open dataset using Xarray
df_sat = xr.open_dataset(cwd_system_wide+'/data/subset_0_of_Mosaic_L2C2RCC_NA_S3A_OL_1_EFR_20200806__NT.nc')
# Create a mask where the pixel_classif_flags is equal to -32768
mask = df_sat['pixel_classif_flags'] == -32768
# Filter the data using the mask
filtered_sat = df_sat.where(mask, drop=True)

# Define original sensor + AC used
sensor_name = 'OLCI_c2rcc'

# Calculate Rrs and generate a modified dataset with new Rrs bands
olci_c2rcc_bands = sensor_meta_info.get_sensor_config(sensor_name)
filtered_sat_rrs = sensor_meta_info.calculate_rrs(filtered_sat, olci_c2rcc_bands)

# Use the Rrs sensor bands
sensor_rrs ='OLCI_c2rcc_rrs'

# Process dataset 
#input_data_owts = data_pre_processing.owt_flagging(owt_rrs = owts_iw, input_dataset=filtered_sat_rrs, sensor=sensor_rrs, data_type='satellite')

NameError: name 'cwd_system_wide' is not defined

In [3]:
cd \\eawag\userdata\werthemo\Desktop\test_bnn_image

\\eawag\userdata\werthemo\Desktop\test_bnn_image


In [7]:
df_sat = xr.open_dataset('subset_0_of_Mosaic_L2C2RCC_NA_S3A_OL_1_EFR_20200806__NT.nc')

In [12]:
# Create a mask where the pixel_classif_flags is equal to -32768
mask = df_sat['pixel_classif_flags'] == -32768
# Filter the data using the mask
filtered_sat = df_sat.where(mask, drop=True)

In [16]:
# Define original sensor + AC used
sensor_name = 'OLCI_c2rcc'

# Calculate Rrs and generate a modified dataset with new Rrs bands
olci_c2rcc_bands = get_sensor_config(sensor_name)
filtered_sat_rrs = calculate_rrs(filtered_sat, olci_c2rcc_bands)

# Use the Rrs sensor bands
sensor_rrs ='OLCI_c2rcc_rrs'

Band rhow_2 was divided by pi and stored as Rrs_2.
Band rhow_3 was divided by pi and stored as Rrs_3.
Band rhow_4 was divided by pi and stored as Rrs_4.
Band rhow_5 was divided by pi and stored as Rrs_5.
Band rhow_6 was divided by pi and stored as Rrs_6.
Band rhow_7 was divided by pi and stored as Rrs_7.
Band rhow_8 was divided by pi and stored as Rrs_8.
Band rhow_9 was divided by pi and stored as Rrs_9.
Band rhow_10 was divided by pi and stored as Rrs_10.
Band rhow_11 was divided by pi and stored as Rrs_11.
Band rhow_12 was divided by pi and stored as Rrs_12.
Band rhow_16 was divided by pi and stored as Rrs_16.


In [ ]:
input_data_owts = owt_flagging(owt_rrs = owts_iw, input_dataset=filtered_sat_rrs, sensor=sensor_rrs, data_type='satellite')

In [ ]:
def calc_max_owt(rrs_spectrum, owt_array: np.ndarray, data_type: str) -> int:
    """
    Calculate the maximum OWT class for a given Rrs spectrum.
    
    Args:
        rrs_spectrum (pandas.Series or xarray.DataArray): A one-dimensional pandas Series or a two-dimensional xarray DataArray containing the Rrs spectrum to calculate the maximum OWT class for.
        owt_array (np.ndarray): A two-dimensional array containing the OWTs to use for the calculation.
        data_type (str): A string specifying the type of rrs_spectrum. Either 'in_situ' or 'satellite'. 
    
    Returns:
        int or xarray.DataArray: The maximum OWT class for the given spectrum. If rrs_spectrum is a pandas Series, this is a single int. If rrs_spectrum is an xarray DataArray, this is an xarray DataArray of ints.
    
    Raises:
        ValueError: If data_type is not one of 'in_situ' or 'satellite'.
    """

    if data_type not in ['in_situ', 'satellite']:
        raise ValueError("Invalid data_type. Expected one of: 'in_situ', 'satellite'")

    # Calculate the spectral angle (SA) for each OWT in the OWT array using the given Rrs spectrum
    if data_type == 'in_situ':
        all_sa = np.apply_along_axis(cal_sa, 1, owt_array, rrs_spectrum=rrs_spectrum, data_type=data_type)
        # Find the maximum SA and corresponding OWT class, + 1 because python starts counting at 0
        owt_class = np.argmax(all_sa) + 1

    elif data_type == 'satellite':
        all_sa = rrs_spectrum.apply(lambda data_array: np.apply_along_axis(cal_sa, 1, owt_array, rrs_spectrum=data_array, data_type=data_type), dim=('band'))
        # Find the maximum SA and corresponding OWT class, + 1 because python starts counting at 0
        owt_class = all_sa.argmax(dim=('band')) + 1
        
    return owt_class

def owt_flagging(owt_rrs: pd.DataFrame, input_dataset, sensor: str, data_type: str) -> Union[pd.DataFrame, xr.Dataset]:
    """
    Calculates an OWT per observation and the OWT quality flag (inside (0) and outside (1) of the application scope, respectively).
    
    Args:
        owt_rrs (pandas.DataFrame): A data frame containing the mean standardised OWTs of Spyrakos et al. (2018).
        input_dataset (pandas.DataFrame or xr.Dataset): A data frame or xarray dataset to produce the OWTs for.
        sensor (str): A sensor configuration, one of ['OLCI_all', 'OLCI_polymer', 'MSI_s2a', 'MSI_s2b'].
        data_type (str): A string specifying the type of input_dataset. Either 'in_situ' or 'satellite'. 
    
    Returns:
        pandas.DataFrame or xr.Dataset: If 'in_situ' data, returns the input pandas DataFrame with 'owt_class' and 'owt_flag' columns. If 'satellite' data, returns the input xarray Dataset with 'owt_class' and 'owt_flag' variables. The return is not standardised, this is just undertaken for the calculation of the OWTs.
    
    Raises:
        ValueError: If the input dataset contains NaN values in the input sensor columns.
        ValueError: If data_type is not one of 'in_situ' or 'satellite'.
        
    """

    if data_type not in ['in_situ', 'satellite']:
        raise ValueError("Invalid data_type. Expected one of: 'in_situ', 'satellite'")
    
    if data_type == 'in_situ':
        # Execute original function for pandas DataFrame
 
        # Select the sensor configuration for the given sensor
        sensor_bands = meta.get_sensor_config(sensor, input_dataset)
        
        # If it does, raise a ValueError - there should be no NaN values in any Rrs band in an in situ data file
        if input_dataset[sensor_bands].isnull().values.any():
            raise ValueError('The input dataset contains NaN values in one or more of the sensor columns specified in sensor_meta_info.py. Please treat/remove the corresponding observation.')

        # Select the specified columns from the owt_rrs and input_dataset data frames
        input_dataset_sel = input_dataset[sensor_bands]

        # Standardise the values in each row of input_dataset_sel
        input_dataset_standardised = input_dataset_sel.apply(lambda row: standardise_rrs(row, data_type), axis=1)

        # Calculate the maximum OWT class for each row in the input dataset
        owt_cols = ['wl'] + sensor_bands
        owt_sel = owt_rrs[owt_cols]
        owt_array = owt_sel.iloc[:, 1:].values
        max_owt_classes = np.apply_along_axis(lambda x: calc_max_owt(x, owt_array=owt_array, data_type=data_type), 1, input_dataset_standardised.values)

        # These are the valid OWTs of the manuscript the BNNs were designed for:
        bnn_owts = [2, 3, 4, 5, 9]

        # For each maximum OWT membership, determine the corresponding OWT flag
        owt_flag = 1 - np.isin(max_owt_classes, bnn_owts).astype(int)

        # Add the maximum OWT membership and OWT flag as columns to the input dataset
        input_dataset['owt_class'] = max_owt_classes
        input_dataset['owt_flag'] = owt_flag

        print('OWT flagging complete. New OWT columns "owt_class" and "owt_flag" added to input dataframe.')
        return input_dataset
    
    elif data_type == 'satellite':
         # Select the sensor configuration for the given sensor
        sensor_bands = meta.get_sensor_config(sensor)
    
        #  Skip those pixels and proceed with the calculations for the remaining pixels.
        #  The structure of the original dataset will be preserved since the NaN pixels are not dropped, but ignored during the computations.

        for band in sensor_bands:
            print(band)
            if np.isnan(input_dataset[band]).any():
                print(f'Skipping band {band} due to NaN values.')
                continue
            
        # Create a list of data arrays for each band in the sensor configuration
        list_of_data_arrays = [input_dataset[band] for band in sensor_bands]
        # Combine the list of data arrays into a new data array
        data_array_combined = xr.concat(list_of_data_arrays, dim='band')
        # Stack 'lat' and 'lon' into a single multi-index
        data_array_combined_stacked = data_array_combined.stack(pixel=('lat', 'lon'))
        # Group by 'pixel' and apply the standardise_rrs function to each group
        input_dataset_standardised_stacked = data_array_combined_stacked.groupby('pixel').apply(standardise_rrs, data_type='satellite')
        print('Standardising all valid Rrs spectra.')
        # Unstack the 'pixel' multi-index back to 'lat' and 'lon'
        input_dataset_standardised = input_dataset_standardised_stacked.unstack('pixel')

        # Split the standardised data array back into separate bands and update the original dataset
        for i, band in enumerate(sensor_bands):
            input_dataset[band] = input_dataset_standardised.isel(band=i)

        print('Standardisation complete.')

        # Select OWT columns for calculating the maximum OWT membership
        # The OWT dataframe is already standardised and as such both data sources share the same standardisation

        # OWT band selection is independent of AC, but of sensor
        # If 'OLCI' is in 'sensor', change 'sensor' to 'OLCI_all'
        if 'OLCI' in sensor:
            sensor = 'OLCI_all'
    
        # Fetch the sensor configuration based on the updated 'sensor'
        sensor_bands = meta.get_sensor_config(sensor)
        
        owt_cols = ['wl'] + sensor_bands
        # Select the specified columns from the owt_rrs data frame
        owt_sel = owt_rrs[owt_cols]
        owt_array = owt_sel.iloc[:, 1:].values

        # calculate OWT using the standardised pixels (input_dataset_standardised)
        max_owt_classes = input_dataset_standardised.apply(lambda x: calc_max_owt(x, owt_array=owt_array, data_type=data_type), dim=('lat', 'lon'))

        # These are the valid OWTs of the manuscript the BNNs were designed for:
        bnn_owts = [2, 3, 4, 5, 9]

        # For each maximum OWT membership, determine the corresponding OWT flag
        # Using xarray's where method to handle multi-dimensional array
        owt_flag = xr.where(np.isin(max_owt_classes, bnn_owts), 0, 1)

        # Add the maximum OWT membership and OWT flag as variables to the input dataset
        input_dataset['owt_class'] = max_owt_classes
        input_dataset['owt_flag'] = owt_flag

        print('OWT flagging complete. New OWT variables "owt_class" and "owt_flag" added to input dataset.')
        return input_dataset
